In [3]:
import os
import subprocess
import shutil
import pandas
from tkinter import simpledialog
import tkinter as tk

In [ ]:
#Inicialização de variáveis
patients = [];
errors = [];
chrome_path = 'C:\\Program Files (x86)\\Google\\Chrome\\Application\\chrome.exe'
#start = input("Iniciar em qual paciente?");

In [ ]:
#Leitura dos dataframes pré-existentes
#surgeryFrame = pandas.read_excel("CIRURGIAS.xlsx")
data = pandas.read_excel("Output.xlsx")

## ATENÇÃO: AQUI AINDA FALTA FILTRAR OS PACIENTES QUE JÁ ESTÃO NO XLSX DE SAÍDA E SELECIONAR O PRÓXIMO PACIENTE ALEATORIAMENTE

In [ ]:
#Redirecionamento para o diretório de pacientes
os.chdir("./Pacientes")

#Coletando nomes de pacientes
for root, directory, files in os.walk("."):
    if(root != "."):
        patients.append(root[2:])

patients = sorted(patients)

In [ ]:
def storeFiles():
    files = []
    for root, directory, file in os.walk('.'):
        pass
        #files.append(file)
    return file

In [ ]:
def createPatientDF(name):
    cols = ['Nome', 'Registro', 'Data da Cirurgia', 'Olho', 'Dioptria', 'Lente', 'Modelo', 'Data Pré-op', 'Esférico Pré-op',
            'Cilindro Pré-op', 'Eixo Pré-op', 'Data Pós-op', 'Esférico Pós-op', 'Cilindro Pós-op', 'Eixo Pós-op']
    patientDF = pandas.DataFrame(columns = cols)
    return patientDF

In [ ]:
def getSurgeries(patientDF, files):
    for file in files:
        if 'PRONTU' in file:
            subprocess.Popen([chrome_path, file])

In [ ]:
#Iterando pelos pacientes
for patient in patients:
    os.chdir("./" + patient)
    
    files = storeFiles()

In [6]:
def salvarDados():
    pass

In [7]:
def voltar():
    pass

In [8]:
def avancar():
    pass

In [9]:
def salvarErro():
    pass

In [10]:
def sair():
    pass

In [21]:
root = tk.Tk()

tk.Label(root, text="Registro").grid(row=0)
tk.Label(root, text="Data da Cirurgia (apenas números)").grid(row=1)
tk.Label(root, text="Olho").grid(row=2)
tk.Label(root, text="Dioptria").grid(row=3)
tk.Label(root, text="Modelo da Lente").grid(row=4)

reg = tk.Entry(root)
data = tk.Entry(root)
olho = tk.Entry(root)
dioptria = tk.Entry(root)
modelo = tk.Entry(root)

reg.grid(row=0, column=1)
data.grid(row=1, column=1)
olho.grid(row=2, column=1)
dioptria.grid(row=3, column=1)
modelo.grid(row=4, column=1)

tk.Button(root, text="Anterior", command=voltar()).grid(row=5, column=0, pady=4)
tk.Button(root, text="Ok", command=salvarDados()).grid(row=5, column=1, pady=4)
tk.Button(root, text="Próximo", command=avancar()).grid(row=5, column=2, pady=4)
tk.Button(root, text="Erro", command=salvarErro()).grid(row=6, column=0, pady=4)
tk.Button(root, text="Sair", command=sair()).grid(row=6, column=1)

root.mainloop()

## LIXO

#Processando DataFrames
def upperCaseFrame():
    names = []
    for patient in surgeryFrame["PACIENTE"]:
        names.append(patient.upper())
    surgeryFrame["PACIENTE"] = names

#upperCaseFrame()

def createPatientDF(name):
    '''if surgeryFrame["PACIENTE"].str.contains(name).sum() > 0:
        #Inicialização do Dataframe
        cols = list(surgeryFrame.columns)
        newCols = ["Data pré-op".upper(), "Esférico pré-op".upper(), "Cilíndrico  pré-op".upper(), "Eixo  pré-op".upper(), 
                                          "Data pós-op".upper(), "Esférico pós-op".upper(), "Cilíndrico  pós-op".upper(), "Eixo  pós-op".upper()]
        cols.extend(newCols)
        patientDF = pandas.DataFrame(columns = cols)
        
        #Copiando informações do Dataframe original
        return patientDF
    else:
        errors.append(name)'''